In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

In [2]:
df = pd.read_csv("D:/My Projects/experiment-uplifting engine/data/ecommerce_product_dataset.csv")
df.head()

,ProductID,ProductName,Category,Price,Rating,NumReviews,StockQuantity,Discount,Sales,DateAdded,City
0,1,Headphones,Electronics,400.31,1.7,3772,20,0.08,466,2023-11-10,Albuquerque
1,2,Headphones,Electronics,235.03,2.3,2919,663,0.33,1332,2023-09-28,Queens
2,3,Smartwatch,Electronics,417.90,1.8,1184,459,0.31,252,2023-10-20,Austin
3,4,Smartphone,Electronics,152.70,3.4,2047,475,0.49,1806,2023-12-05,Dallas
4,5,Laptop,Electronics,394.74,1.8,1267,831,0.23,1508,2023-08-10,Tucson


In [3]:
df.info()
df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ProductID      1000 non-null   int64  
 1   ProductName    1000 non-null   object 
 2   Category       1000 non-null   object 
 3   Price          1000 non-null   float64
 4   Rating         1000 non-null   float64
 5   NumReviews     1000 non-null   int64  
 6   StockQuantity  1000 non-null   int64  
 7   Discount       1000 non-null   float64
 8   Sales          1000 non-null   int64  
 9   DateAdded      1000 non-null   object 
 10  City           1000 non-null   object 
dtypes: float64(3), int64(4), object(4)
memory usage: 86.1+ KB


,ProductID,ProductName,Category,Price,Rating,NumReviews,StockQuantity,Discount,Sales,DateAdded,City
count,1000.000000,1000,1000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000,1000
unique,NaN,100,25,NaN,NaN,NaN,NaN,NaN,NaN,346,101
top,NaN,Biography,Electronics,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-09,Palm Bay
freq,NaN,22,40,NaN,NaN,NaN,NaN,NaN,NaN,7,17
mean,500.500000,NaN,NaN,253.77551,3.025600,2498.753000,495.395000,0.251640,1011.037000,NaN,NaN
std,288.819436,NaN,NaN,141.40362,1.151004,1463.241871,292.799253,0.146455,582.113466,NaN,NaN
min,1.000000,NaN,NaN,10.11000,1.000000,3.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,250.750000,NaN,NaN,133.09250,2.100000,1201.750000,241.750000,0.130000,502.000000,NaN,NaN
50%,500.500000,NaN,NaN,251.31000,3.100000,2476.000000,505.000000,0.250000,998.000000,NaN,NaN
75%,750.250000,NaN,NaN,375.82750,4.000000,3797.500000,743.500000,0.380000,1540.000000,NaN,NaN


In [5]:
# Example engineered features
if 'prior_purchases' not in df.columns:
    # Fallback: approximate prior purchases from review volume
    df['prior_purchases'] = (df['NumReviews'] / 500).round().astype(int)

df['new_user'] = np.where(df['prior_purchases'] == 0, 1, 0)
df['high_value_user'] = np.where(df['prior_purchases'] > 5, 1, 0)
df[['prior_purchases', 'new_user', 'high_value_user']].head()


,prior_purchases,new_user,high_value_user
0,8,0,1
1,6,0,1
2,2,0,0
3,4,0,0
4,3,0,0


In [7]:
df['treatment'] = np.random.binomial(1, 0.5, len(df))

baseline_prob = 0.08

if 'device' not in df.columns:
    # Fallback: synthesize device type for simulation
    df['device'] = np.random.choice(['mobile', 'desktop'], size=len(df), p=[0.6, 0.4])

df['treatment_effect'] = (
    0.03 * df['new_user'] +
    0.02 * (df['device'] == 'mobile').astype(int) -
    0.01 * df['high_value_user']
)

df['conversion_prob'] = baseline_prob + df['treatment'] * df['treatment_effect']
df['conversion_prob'] = df['conversion_prob'].clip(0, 0.5)

df['converted'] = np.random.binomial(1, df['conversion_prob'])
df[['device', 'treatment', 'treatment_effect', 'conversion_prob', 'converted']].head()


,device,treatment,treatment_effect,conversion_prob,converted
0,mobile,0,0.01,0.08,0
1,mobile,1,0.01,0.09,0
2,desktop,1,0.00,0.08,0
3,mobile,1,0.02,0.10,0
4,mobile,1,0.02,0.10,0


In [8]:
df.to_csv("../data/experiment_simulated.csv", index=False)